<a href="https://colab.research.google.com/github/skywalker0803r/CFB/blob/main/CFB_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np
import joblib
import warnings
warnings.simplefilter('ignore')
from sklearn.metrics import r2_score ,mean_squared_error
from sklearn.linear_model import Lasso
from xgboost import XGBRegressor
from tqdm import tqdm_notebook as tqdm
import pandas as pd
from sklearn.svm import SVR

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# 載入數據

In [10]:
data = joblib.load('/content/drive/MyDrive/脫硫劑優化改善/dataset.pkl')
b = data['y']
A = data['X'].reshape(len(b),-1)
print(A.shape,b.shape)

(54187, 580) (54187, 1)


# 訓練測試切分

In [11]:
sp = 50000
A_train = A[:sp]
b_train = b[:sp]
A_test = A[sp:]
b_test = b[sp:]

# 評估函數

In [12]:
def evaluate(y_real,y_pred,name):
    score = pd.DataFrame(index = ['表現'],columns = ['R2','MSE','MAPE'])
    score['R2'] = r2_score(y_real,y_pred)
    score['MSE'] = mean_squared_error(y_real,y_pred)
    score['MAPE'] = mape(y_real,y_pred)
    score.index.name = name
    return score

# lasso 參數最佳化

In [15]:
params = {}
s = 2.907000e-08
t = s * 0.01
e = s + 10*t
for a in tqdm(np.arange(s,e,t)):
    model = Lasso(a).fit(A_train,b_train)
    y_pred = data['mm_y'].inverse_transform(model.predict(A_test).reshape(-1,1))
    y_real = data['mm_y'].inverse_transform(b_test)
    params[a] = evaluate(y_real,y_pred,'bababa')

  0%|          | 0/11 [00:00<?, ?it/s]

In [16]:
temp = pd.DataFrame([ i.R2.values[0] for i in params.values()],index=params.keys(),columns=['R2'])
temp.index.name = 'lambda'
temp

,R2
lambda,
2.907000e-08,0.999966
2.936070e-08,0.999966
2.965140e-08,0.999968
2.994210e-08,0.999968
3.023280e-08,0.999967
3.052350e-08,0.999967
3.081420e-08,0.999967
3.110490e-08,0.999967
3.139560e-08,0.999967


In [17]:
a = temp.index[np.argmax(temp)]
a

np.float64(2.96514e-08)

# 建模

In [18]:
model = Lasso(a).fit(A_train,b_train)

In [19]:
data['x_col']

Index(['MLUT4_AIA792101A', 'MLUT4_AIA792101B', 'MLUT4_AIC-232B',
       'MLUT4_AT-232A', 'MLUT4_AT-237', 'MLUT4_AT-240A', 'MLUT4_FIC-231A',
       'MLUT4_FIC-231B', 'MLUT4_FIC-231C', 'MLUT4_FIC-233', 'MLUT4_FIQ-2BTCF',
       'MLUT4_FQ-205', 'MLUT4_FQ-239', 'MLUT4_FT-232', 'MLUT4_FT-792003',
       'MLUT4_FT-792020', 'MLUT4_FT-951AA', 'MLUT4_FT-951AB', 'MLUT4_FT-956',
       'MLUT4_FT-957', 'MLUT4_FT-V004', 'MLUT4_IT-E7925A', 'MLUT4_IT-E7925B',
       'MLUT4_PDIC-233', 'MLUT4_PDT-231', 'MLUT4_PDT-232', 'MLUT4_PIC-233',
       'MLUT4_PT-231', 'MLUT4_RQ-2BTLS', 'MLUT4_RT-031', 'MLUT4_RT-041',
       'MLUT4_RT-051', 'MLUT4_RT-081', 'MLUT4_RT-091', 'MLUT4_TE-251A',
       'MLUT4_TE-251B', 'MLUT4_TE-251C', 'MLUT4_TE-251D', 'MLUT4_TE-251E',
       'MLUT4_TE-251F', 'MLUT4_TE-251G', 'MLUT4_TE-251H', 'MLUT4_TE-251I',
       'MLUT4_TE-252A', 'MLUT4_TE-252B', 'MLUT4_TE-252C', 'MLUT4_TE-252D',
       'MLUT4_TE-252E', 'MLUT4_TE-252F', 'MLUT4_TE-252G', 'MLUT4_TE-252H',
       'MLUT4_TE-252I', 'MLUT4

# 預測

In [20]:
y_pred = data['mm_y'].inverse_transform(model.predict(A_test).reshape(-1,1)).reshape(-1)
y_real = data['mm_y'].inverse_transform(b_test).reshape(-1)
evaluate(y_real,y_pred,'煙囪SOx濃度')

,R2,MSE,MAPE
煙囪SOx濃度,,,
表現,0.999968,0.000455,0.184683


In [32]:
import pandas as pd
import numpy as np

# 取得特徵係數
coefficients = model.coef_


result = pd.DataFrame(coefficients.reshape(-1,len(data['x_col'])))#.style.background_gradient()
result.style.background_gradient()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57
0,0.000000,0.000353,0.000151,-0.000215,0.000065,0.000022,0.000896,0.000143,0.000225,-0.002288,0.000098,0.000145,-0.000059,0.000000,0.000329,-0.000576,-0.000000,-0.000000,-0.000015,-0.000185,0.001004,0.000029,-0.000012,-0.000000,0.009636,0.003315,0.001715,-0.005319,-0.000023,-0.000000,-0.000000,0.000000,-0.000068,-0.000073,-0.002228,-0.000702,-0.000245,-0.000000,-0.000000,-0.001022,0.000000,-0.000737,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,-0.000535,0.000000,-0.000929,-0.002219,-0.000000,-0.002407,0.000718,0.000000,-0.000000,-0.000000,0.000000
1,-0.000000,-0.000021,-0.000369,0.000277,0.000000,-0.000014,-0.000010,0.000782,-0.000099,0.001311,-0.001313,-0.000186,0.000000,-0.000000,-0.000311,-0.000029,0.000000,0.000027,0.000144,0.000199,0.000030,0.000037,-0.000014,0.000000,0.000704,0.001258,-0.000526,-0.002228,-0.000000,-0.000000,0.000000,-0.000000,-0.000000,0.000091,0.000522,0.002363,0.000000,0.002470,0.001973,0.000951,0.001114,0.000092,0.000373,-0.003564,-0.000000,-0.003008,-0.001184,-0.000000,0.000000,-0.000885,-0.000001,-0.000000,0.000114,-0.000008,0.000000,-0.000000,-0.000000,0.000000
2,0.000097,-0.000224,-0.000000,0.000071,-0.000358,-0.000000,0.000938,0.000319,0.000524,0.002057,-0.000778,-0.000001,-0.000000,-0.000049,-0.000063,-0.000047,-0.000000,0.000127,-0.000067,0.000014,0.000358,0.000000,-0.000007,-0.000000,0.000000,0.000132,0.000000,-0.001373,-0.000015,-0.000000,0.000000,-0.000000,-0.000000,-0.000032,0.001127,0.000000,-0.000000,0.000249,0.000264,0.000989,0.000878,0.000000,0.000125,0.000000,0.000035,0.000000,-0.000000,0.000000,0.000004,0.000000,0.000000,0.000000,0.001351,-0.000000,-0.000000,-0.000000,0.000000,-0.000000
3,-0.000000,-0.000219,0.000000,0.000012,-0.001039,-0.000023,0.000598,0.000806,0.000661,0.000590,-0.000509,-0.000183,-0.000000,-0.000168,-0.000000,0.000053,-0.000000,0.000039,0.000003,0.000088,-0.000074,-0.000000,-0.000000,0.000000,0.000000,-0.001002,0.000000,-0.001084,0.000000,-0.000000,0.000000,-0.000000,0.000013,0.000004,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,-0.000407,-0.000000,-0.000000,0.000000,-0.000000,-0.000177,0.000000,0.000000,-0.000000,-0.000000,-0.000000,0.002936,-0.000505,-0.000000,-0.000000,0.000000,0.000000
4,0.000000,-0.000317,0.000000,0.000003,-0.001542,-0.000000,0.000578,0.000604,0.000768,-0.000000,-0.000947,0.000000,0.000000,-0.000000,-0.000000,0.000256,-0.000000,0.000000,-0.000080,0.000000,-0.000219,-0.000000,-0.000017,0.000000,0.000180,-0.001121,0.000186,-0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,0.000004,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000,-0.000289,0.000000,0.000000,-0.000000,0.000000,0.000000,0.003137,-0.000140,-0.000000,0.000000,0.000000,0.000000
5,-0.000000,-0.000485,0.000068,-0.000058,-0.001285,0.000018,0.000338,0.000162,0.000396,0.000000,-0.001135,0.000020,0.000060,-0.000000,-0.000000,0.000265,-0.000000,0.000000,-0.000000,0.000065,-0.000594,0.000000,-0.000007,0.000000,-0.001292,-0.001784,0.000573,-0.000278,0.000000,-0.000000,0.000000,-0.000000,0.000036,0.000002,0.000055,-0.000029,0.000621,0.000000,-0.000000,0.000000,-0.000527,-0.000628,-0.000285,-0.000000,0.000000,-0.000097,-0.001271,0.000000,0.000000,-0.000590,0.001620,0.000000,0.001582,0.000000,-0.000000,0.000000,0.000000,-0.000000
6,0.000000,-0.000035,0.000158,-0.000098,-0.000000,0.000005,-0.000088,-0.000515,-0.000581,0.000116,-0.000727,0.000122,0.000009,-0.000000,-0.000000,0.000333,-0.000046,0.000000,-0.000000,-0.000380,-0.000667,0.000050,-0.000058,-0.000000,-0.004826,-0.002818,0.000705,-0.000000,0.000011,-0.000000,0.000096,-0.000000,0.000049,0.000000,0.000000,-0.000004,0.000016,0.000000,-0.000591,-0.000212,-0.004720,-0.000221,-0.002721,-0.000125,-0.000000,-0.000145,-0.000101,0.001912,0.000000,-0.000000,0.004148,0.000000,0.000050,-0.000661,-0.

In [35]:
data['x_col']

Index(['MLUT4_AIA792101A', 'MLUT4_AIA792101B', 'MLUT4_AIC-232B',
       'MLUT4_AT-232A', 'MLUT4_AT-237', 'MLUT4_AT-240A', 'MLUT4_FIC-231A',
       'MLUT4_FIC-231B', 'MLUT4_FIC-231C', 'MLUT4_FIC-233', 'MLUT4_FIQ-2BTCF',
       'MLUT4_FQ-205', 'MLUT4_FQ-239', 'MLUT4_FT-232', 'MLUT4_FT-792003',
       'MLUT4_FT-792020', 'MLUT4_FT-951AA', 'MLUT4_FT-951AB', 'MLUT4_FT-956',
       'MLUT4_FT-957', 'MLUT4_FT-V004', 'MLUT4_IT-E7925A', 'MLUT4_IT-E7925B',
       'MLUT4_PDIC-233', 'MLUT4_PDT-231', 'MLUT4_PDT-232', 'MLUT4_PIC-233',
       'MLUT4_PT-231', 'MLUT4_RQ-2BTLS', 'MLUT4_RT-031', 'MLUT4_RT-041',
       'MLUT4_RT-051', 'MLUT4_RT-081', 'MLUT4_RT-091', 'MLUT4_TE-251A',
       'MLUT4_TE-251B', 'MLUT4_TE-251C', 'MLUT4_TE-251D', 'MLUT4_TE-251E',
       'MLUT4_TE-251F', 'MLUT4_TE-251G', 'MLUT4_TE-251H', 'MLUT4_TE-251I',
       'MLUT4_TE-252A', 'MLUT4_TE-252B', 'MLUT4_TE-252C', 'MLUT4_TE-252D',
       'MLUT4_TE-252E', 'MLUT4_TE-252F', 'MLUT4_TE-252G', 'MLUT4_TE-252H',
       'MLUT4_TE-252I', 'MLUT4

In [33]:
max_idx = result.stack().idxmax()  # 回傳 (row_index, column_name)

print("最大值位置 (row, column):", max_idx)
print("最大值是:", result.loc[max_idx])

最大值位置 (row, column): (9, 5)
最大值是: 1.0000950759936869
